# Extract - Data Loading

This notebook handles the extraction of raw data from source files.


In [ ]:
import pandas as pd
import numpy as np
import warnings
np.random.seed(34)
warnings.filterwarnings('ignore')


In [ ]:
# Define column names
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]
col_names = index_names + setting_names + sensor_names


In [ ]:
# Extract raw data from source files
dftrain = pd.read_csv('train_FD001.txt',sep='\s+',header=None,index_col=False,names=col_names)
dfvalid = pd.read_csv('test_FD001.txt',sep='\s+',header=None,index_col=False,names=col_names)
y_valid = pd.read_csv('RUL_FD001.txt',sep='\s+',header=None,index_col=False,names=['RUL'])


In [ ]:
# Create copies for processing
train = dftrain.copy()
valid = dfvalid.copy()


In [ ]:
# Verify extraction
print('Shape of the train dataset : ',train.shape)
print('Shape of the validation dataset : ',valid.shape)
print('Shape of the RUL validation dataset : ',y_valid.shape)


In [ ]:
# Save extracted data for next stage (Transform)
import os
train.to_csv('train_extracted.csv', index=False)
valid.to_csv('valid_extracted.csv', index=False)
y_valid.to_csv('y_valid_extracted.csv', index=False)
print("Extracted data saved for transformation stage")
